# Corporación Favorita Grocery Sales Forecasting

Authors: shiying Chen, Jianjun Du,  Bo Huang, Yanyan Zhu, 

Features: big data, 120 million records; time series; regressions; clusering

Part I: data preprocess, descriptive data analysis, feature engineering

Part II: model selection, tuning, and evaluation

## part I: data preprocess, descriptive data analysis, feature engineer

### 1.1 data input and preprocess

In [ ]:
#  Corporación Favorita Grocery Sales Forecasting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
# read in data, and made some modification to make them smaller
# reference: https://www.kaggle.com/heyt0ny/read-data-for-low-memory-usage
def loaddata(filename):
    types = {'id': 'int64',
             'item_nbr': 'int32',
             'store_nbr': 'int16',
             'unit_sales': 'float32',
             'onpromotion': bool,
    }
    data = pd.read_csv(filename, dtype=types)
    data['onpromotion'].fillna(False, inplace = True)
    data['onpromotion'] = data['onpromotion'].map({False : 0, True : 1})
    data['onpromotion'] = data['onpromotion'].astype('int8')
    return data

train=loaddata("train.csv")

In [ ]:
# detach the date column to year (2013, 2014,..), day of the week(monday,tuesday,..), month(january, February, et..)
DateIndex=pd.DatetimeIndex(train['date'])
train['year']=DateIndex.year
train['month']=DateIndex.month
train['dayofweek']=DateIndex.weekday

In [ ]:
stores=pd.read_csv("stores.csv")
holidays=pd.read_csv("holidays_events.csv")
items=pd.read_csv("items.csv")
oilprice=pd.read_csv("oil.csv")
transactions=pd.read_csv("transactions.csv")

### 1.2 descriptive data analysis

In [ ]:
# How many items are sold by the company?
train.item_nbr.nunique()


In [ ]:
# How many stores owned by the company?
train.store_nbr.nunique()

In [ ]:
# Is there any difference for each day of week?
grouped=train[['dayofweek','unit_sales']].groupby('dayofweek')
sales=grouped.sum()
plt.plot(sales)
plt.title("unit sales for each day of a week")
plt.xlabel("day of week, 0 is Sunday, 6 is Saturda")
plt.ylabel("Sum of unit sales by week day")
plt.show()

# The figure shows that Friday and Saturday have the biggest sales, and Wednesday has the smmallest sales

In [ ]:
# Is there any difference for each month?
grouped=train[['month','unit_sales']].groupby('month')
sales=grouped.sum()
plt.plot(sales)
plt.title("unit sales for each month")
plt.xlabel("Month")
plt.ylabel("Sum of unit sales by month")
plt.show()

# July has the biggest sale

In [ ]:
# Is there any difference for each year?
grouped=train[['year','unit_sales']].groupby('year')
sales=grouped.sum()
plt.plot(sales)
plt.title("unit sales from 2013 to 2017")
plt.xlabel("year")
plt.ylabel("Sum of unit sales by year")
plt.show()

# The sales is decreasing from 2016 to 2017, that is because the data only cover part of 2017.

In [ ]:
# What are the sales per stores?
# What are the locations of the top 10 stores
grouped=train[['store_nbr','unit_sales']].groupby('store_nbr')
sales=grouped.sum().sort_values('unit_sales',ascending=False)
sales['store_nbr']=sales.index
pd.merge(sales,stores,on='store_nbr').head(10)

# Quito, Pichincha is the major market for the company

In [ ]:
# What are the top sold items?
grouped=train[['item_nbr','unit_sales']].groupby('item_nbr')
sales=grouped.sum().sort_values('unit_sales',ascending=False)
sales['item_nbr']=sales.index
sales.head(10)

In [ ]:
for i in range(55):
    data=train[train.store_nbr==i]
    path="store_"+str(i)+".csv"
    data.to_csv(path)